# Introduction: Business Problem

In this project we will try to find an optimal location to open a. Specifically, this report will be targeted to stakeholders interested in opening an Coffee House in Cluj-Napoca, Romania.
Since there are lots of Coffee Houses in Cluj-Napoca we will try to detect locations that are not already crowded with Coffee Houses. We would also prefer locations as close to city center as possible.

# Data

Based on definition of our problem, factors that will influence our decision are:

###### •	number of existing Coffee Houses/places in the neighborhood

###### •	number of and distance to Coffee Houses in the neighborhood, if any

###### •	distance of neighborhood from city center

We decided to a radius determined approach since we are only interested in finding out the places near city center. In order to get our stakeholders a clearer view of thing we opted to get 3 sets of data, each further from the city center than the other.

The city center is located at: 46.769379, 23.5899542 also known as „Piata Unirii / Union Square” and it is the ground 0 for cultural, business, relaxation and education activities in the city

The three tiers envisioned are as follows:

- red zone - 750 m from center – most crowded place, roughly 1.75 square kilometers 

- yellow zone – 1250 m from center, roughly 5 square kilometers

- green zone – 2000 m from center, roughly 12.5 square kilometers

### Install and import libraries

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install geopy
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Getting localization data for Cluj-Napoca

In [3]:
address = 'Cluj-Napoca, Cluj, Romania'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

46.769379 23.5899542


Setting up Foursquare API

In [4]:
CLIENT_ID = '1YMM20J1EKW1H02USIKKCIKEGHOZRXP5COVZXO0RFAD2ITR2' # your Foursquare ID
CLIENT_SECRET = 'FZXBPCH0NTC232JEB5RCGACU053PCD2LQXRUF52SEUJOXFRB' # your Foursquare Secret
ACCESS_TOKEN = 'XWJKBPN0TKL4LDEQQ5IXODS0I2SWULMCKBZKWRRKT1IOXWYM' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1YMM20J1EKW1H02USIKKCIKEGHOZRXP5COVZXO0RFAD2ITR2
CLIENT_SECRET:FZXBPCH0NTC232JEB5RCGACU053PCD2LQXRUF52SEUJOXFRB


### The search for red zone

In [5]:
search_query_red = 'Coffee' #searching Coffee places
radius = 500 #500m radius - our red zone
print(search_query_red + ' .... OK!')

Coffee .... OK!


In [6]:
url_red = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query_red, radius, LIMIT)
url_red

'https://api.foursquare.com/v2/venues/search?client_id=1YMM20J1EKW1H02USIKKCIKEGHOZRXP5COVZXO0RFAD2ITR2&client_secret=FZXBPCH0NTC232JEB5RCGACU053PCD2LQXRUF52SEUJOXFRB&ll=46.769379,23.5899542&oauth_token=XWJKBPN0TKL4LDEQQ5IXODS0I2SWULMCKBZKWRRKT1IOXWYM&v=20180604&query=Coffee&radius=500&limit=100'

In [7]:
results_red = requests.get(url_red).json()
results_red

{'meta': {'code': 200, 'requestId': '60caeb39d971343a61677dc0'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '51345ecbe4b041d1dfeded40',
    'name': 'Snack & Coffee',
    'location': {'lat': 46.77135910503546,
     'lng': 23.599048118980512,
     'labeledLatLngs': [{'label': 'display',
       'lat': 46.77135910503546,
       'lng': 23.599048118980512}],
     'distance': 727,
     'cc': 'RO',
     'country': 'România'},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1623911225',
    'hasPerk': False},
   {'id': '5603f924498ec2890ea4b4da',
    'name': "Let's Coffee",
    'location': {'address': 'Str. Constantin Daicoviciu, nr. 8',
     'lat': 46.77111478612739,
     'lng': 23.585028648376465,

In [8]:
# assign relevant part of JSON to venues
venues_red = results_red['response']['venues']

# tranform venues into a dataframe
dataframe_red = json_normalize(venues_red)
dataframe_red.head()

<ipython-input-8-96a1111c8505>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe_red = json_normalize(venues_red)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.address,location.postalCode,location.city,location.state,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,51345ecbe4b041d1dfeded40,Snack & Coffee,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1623911225,False,46.771359,23.599048,"[{'label': 'display', 'lat': 46.77135910503546...",727,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5603f924498ec2890ea4b4da,Let's Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1623911225,False,46.771115,23.585029,"[{'label': 'display', 'lat': 46.77111478612739...",422,RO,România,"Str. Constantin Daicoviciu, nr. 8",400020,Cluj-Napoca,Cluj,"[Str. Constantin Daicoviciu, nr. 8, 400020 Clu...",NaN,NaN,NaN
2,4d3fc9d2c1d4721e2fa70ac7,The Coffee House,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1623911225,False,46.769398,23.590663,"[{'label': 'display', 'lat': 46.769398, 'lng':...",54,RO,România,I. C. Brătianu,NaN,Cluj-Napoca,Cluj,"[I. C. Brătianu (Universității), Cluj-Napoca]",Universității,NaN,NaN
3,53106fce498e43148b52385b,Coffee You Go,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1623911225,False,46.768914,23.590781,"[{'label': 'display', 'lat': 46.76891396084203...",81,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,585a3a790593e17ca0f5480a,Tucano Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1623911225,False,46.769859,23.593079,"[{'label': 'display', 'lat': 46.76985903899314...",244,RO,România,B-dul Eroilor nr. 19,400000,Cluj-Napoca,Cluj,"[B-dul Eroilor nr. 19, 400000 Cluj-Napoca]",NaN,Centru,NaN


In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_red = ['name', 'categories'] + [col for col in dataframe_red.columns if col.startswith('location.')] + ['id']
dataframe_filtered_red = dataframe_red.loc[:, filtered_columns_red]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_red['categories'] = dataframe_filtered_red.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_red.columns = [column.split('.')[-1] for column in dataframe_filtered_red.columns]

dataframe_filtered_red

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,address,postalCode,city,state,formattedAddress,crossStreet,neighborhood,id
0,Snack & Coffee,Café,46.771359,23.599048,"[{'label': 'display', 'lat': 46.77135910503546...",727,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51345ecbe4b041d1dfeded40
1,Let's Coffee,Coffee Shop,46.771115,23.585029,"[{'label': 'display', 'lat': 46.77111478612739...",422,RO,România,"Str. Constantin Daicoviciu, nr. 8",400020,Cluj-Napoca,Cluj,"[Str. Constantin Daicoviciu, nr. 8, 400020 Clu...",NaN,NaN,5603f924498ec2890ea4b4da
2,The Coffee House,Bar,46.769398,23.590663,"[{'label': 'display', 'lat': 46.769398, 'lng':...",54,RO,România,I. C. Brătianu,NaN,Cluj-Napoca,Cluj,"[I. C. Brătianu (Universității), Cluj-Napoca]",Universității,NaN,4d3fc9d2c1d4721e2fa70ac7
3,Coffee You Go,Café,46.768914,23.590781,"[{'label': 'display', 'lat': 46.76891396084203...",81,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53106fce498e43148b52385b
4,Tucano Coffee,Coffee Shop,46.769859,23.593079,"[{'label': 'display', 'lat': 46.76985903899314...",244,RO,România,B-dul Eroilor nr. 19,400000,Cluj-Napoca,Cluj,"[B-dul Eroilor nr. 19, 400000 Cluj-Napoca]",NaN,Centru,585a3a790593e17ca0f5480a
5,Meron - Home of Coffee,Coffee Shop,46.768293,23.589267,"[{'label': 'display', 'lat': 46.76829295132707...",131,RO,România,Str. Napoca nr. 3,900329,Cluj-Napoca,Cluj,"[Str. Napoca nr. 3, 900329 Cluj-Napoca]",NaN,NaN,56641db4498e87cb76ed8837
6,Coffee Addicts,Coffee Shop,46.771816,23.589544,"[{'label': 'display', 'lat': 46.77181588247085...",273,RO,România,Ferdinand Street no 9,400000,Cluj-Napoca,Cluj,"[Ferdinand Street no 9, 400000 Cluj-Napoca]",NaN,NaN,56c4aaf7cd10703f0b5defed
7,Coffee 2 Go,Coffee Shop,46.769740,23.585265,"[{'label': 'display', 'lat': 46.7697396579221,...",359,RO,România,EMIL ISAC,NaN,Cluj-Napoca,Cluj,"[EMIL ISAC (Emil Isac), Cluj-Napoca]",Emil Isac,NaN,574ef6b3498e0e7c7626d67a
8,Coffee Addicts,Coffee Shop,46.771757,23.589818,"[{'label': 'display', 'lat': 46.771757, 'lng':...",264,RO,România,Regele Ferdinand 9,400110,Cluj-Napoca,Cluj,"[Regele Ferdinand 9, 400110 Cluj-Napoca]",NaN,NaN,59520841fd16bb5b31aa1053
9,Coffee To Go,Coffee Shop,46.767489,23.585886,"[{'label': 'display', 'lat': 46.76748861189874...",374,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5236d656498ebc888d2d3eea


In [10]:
dataframe_filtered_red = dataframe_filtered_red.drop(["labeledLatLngs", "cc", "country", "crossStreet", "postalCode", "state"], axis=1)
dataframe_filtered_red

,name,categories,lat,lng,distance,address,city,formattedAddress,neighborhood,id
0,Snack & Coffee,Café,46.771359,23.599048,727,NaN,NaN,NaN,NaN,51345ecbe4b041d1dfeded40
1,Let's Coffee,Coffee Shop,46.771115,23.585029,422,"Str. Constantin Daicoviciu, nr. 8",Cluj-Napoca,"[Str. Constantin Daicoviciu, nr. 8, 400020 Clu...",NaN,5603f924498ec2890ea4b4da
2,The Coffee House,Bar,46.769398,23.590663,54,I. C. Brătianu,Cluj-Napoca,"[I. C. Brătianu (Universității), Cluj-Napoca]",NaN,4d3fc9d2c1d4721e2fa70ac7
3,Coffee You Go,Café,46.768914,23.590781,81,NaN,NaN,NaN,NaN,53106fce498e43148b52385b
4,Tucano Coffee,Coffee Shop,46.769859,23.593079,244,B-dul Eroilor nr. 19,Cluj-Napoca,"[B-dul Eroilor nr. 19, 400000 Cluj-Napoca]",Centru,585a3a790593e17ca0f5480a
5,Meron - Home of Coffee,Coffee Shop,46.768293,23.589267,131,Str. Napoca nr. 3,Cluj-Napoca,"[Str. Napoca nr. 3, 900329 Cluj-Napoca]",NaN,56641db4498e87cb76ed8837
6,Coffee Addicts,Coffee Shop,46.771816,23.589544,273,Ferdinand Street no 9,Cluj-Napoca,"[Ferdinand Street no 9, 400000 Cluj-Napoca]",NaN,56c4aaf7cd10703f0b5defed
7,Coffee 2 Go,Coffee Shop,46.769740,23.585265,359,EMIL ISAC,Cluj-Napoca,"[EMIL ISAC (Emil Isac), Cluj-Napoca]",NaN,574ef6b3498e0e7c7626d67a
8,Coffee Addicts,Coffee Shop,46.771757,23.589818,264,Regele Ferdinand 9,Cluj-Napoca,"[Regele Ferdinand 9, 400110 Cluj-Napoca]",NaN,59520841fd16bb5b31aa1053
9,Coffee To Go,Coffee Shop,46.767489,23.585886,374,NaN,NaN,NaN,NaN,5236d656498ebc888d2d3eea


In [11]:
# create map of Cluj-Napoca - red zone using latitude and longitude values
cluj_red = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(dataframe_filtered_red['lat'], dataframe_filtered_red['lng'], dataframe_filtered_red['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.Circle([latitude, longitude],
                    radius=500).add_to(cluj_red)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cluj_red)  
    
cluj_red

### The search for yellow zone

In [12]:
search_query_yellow = 'Coffee' #searching Coffee places
radius = 750 #750m radius - our yellow zone
print(search_query_yellow + ' .... OK!')

Coffee .... OK!


In [13]:
url_yellow = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query_yellow, radius, LIMIT)
url_yellow

'https://api.foursquare.com/v2/venues/search?client_id=1YMM20J1EKW1H02USIKKCIKEGHOZRXP5COVZXO0RFAD2ITR2&client_secret=FZXBPCH0NTC232JEB5RCGACU053PCD2LQXRUF52SEUJOXFRB&ll=46.769379,23.5899542&oauth_token=XWJKBPN0TKL4LDEQQ5IXODS0I2SWULMCKBZKWRRKT1IOXWYM&v=20180604&query=Coffee&radius=750&limit=100'

In [14]:
results_yellow = requests.get(url_yellow).json()
results_yellow

{'meta': {'code': 200, 'requestId': '60caebe8c6498d362b7c063b'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '51345ecbe4b041d1dfeded40',
    'name': 'Snack & Coffee',
    'location': {'lat': 46.77135910503546,
     'lng': 23.599048118980512,
     'labeledLatLngs': [{'label': 'display',
       'lat': 46.77135910503546,
       'lng': 23.599048118980512}],
     'distance': 727,
     'cc': 'RO',
     'country': 'România'},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1623911400',
    'hasPerk': False},
   {'id': '5603f924498ec2890ea4b4da',
    'name': "Let's Coffee",
    'location': {'address': 'Str. Constantin Daicoviciu, nr. 8',
     'lat': 46.77111478612739,
     'lng': 23.585028648376465,

In [15]:
# assign relevant part of JSON to venues
venues_yellow = results_yellow['response']['venues']

# tranform venues into a dataframe
dataframe_yellow = json_normalize(venues_yellow)
dataframe_yellow.head()

<ipython-input-15-b6fffbeaaef7>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe_yellow = json_normalize(venues_yellow)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.address,location.postalCode,location.city,location.state,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,51345ecbe4b041d1dfeded40,Snack & Coffee,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1623911400,False,46.771359,23.599048,"[{'label': 'display', 'lat': 46.77135910503546...",727,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5603f924498ec2890ea4b4da,Let's Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1623911400,False,46.771115,23.585029,"[{'label': 'display', 'lat': 46.77111478612739...",422,RO,România,"Str. Constantin Daicoviciu, nr. 8",400020,Cluj-Napoca,Cluj,"[Str. Constantin Daicoviciu, nr. 8, 400020 Clu...",NaN,NaN,NaN
2,4d3fc9d2c1d4721e2fa70ac7,The Coffee House,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1623911400,False,46.769398,23.590663,"[{'label': 'display', 'lat': 46.769398, 'lng':...",54,RO,România,I. C. Brătianu,NaN,Cluj-Napoca,Cluj,"[I. C. Brătianu (Universității), Cluj-Napoca]",Universității,NaN,NaN
3,53106fce498e43148b52385b,Coffee You Go,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1623911400,False,46.768914,23.590781,"[{'label': 'display', 'lat': 46.76891396084203...",81,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,585a3a790593e17ca0f5480a,Tucano Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1623911400,False,46.769859,23.593079,"[{'label': 'display', 'lat': 46.76985903899314...",244,RO,România,B-dul Eroilor nr. 19,400000,Cluj-Napoca,Cluj,"[B-dul Eroilor nr. 19, 400000 Cluj-Napoca]",NaN,Centru,NaN


In [16]:
dataframe_yellow.shape

(35, 19)

###### !we see that our yellow zone holds a total of 35 venues, but it includes the ones also included in the red zone, we will need to sort that out later when we create the overall map for this project!

In [17]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_yellow = ['name', 'categories'] + [col for col in dataframe_yellow.columns if col.startswith('location.')] + ['id']
dataframe_filtered_yellow = dataframe_yellow.loc[:, filtered_columns_yellow]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_yellow['categories'] = dataframe_filtered_yellow.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_yellow.columns = [column.split('.')[-1] for column in dataframe_filtered_yellow.columns]

dataframe_filtered_yellow

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,address,postalCode,city,state,formattedAddress,crossStreet,neighborhood,id
0,Snack & Coffee,Café,46.771359,23.599048,"[{'label': 'display', 'lat': 46.77135910503546...",727,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51345ecbe4b041d1dfeded40
1,Let's Coffee,Coffee Shop,46.771115,23.585029,"[{'label': 'display', 'lat': 46.77111478612739...",422,RO,România,"Str. Constantin Daicoviciu, nr. 8",400020,Cluj-Napoca,Cluj,"[Str. Constantin Daicoviciu, nr. 8, 400020 Clu...",NaN,NaN,5603f924498ec2890ea4b4da
2,The Coffee House,Bar,46.769398,23.590663,"[{'label': 'display', 'lat': 46.769398, 'lng':...",54,RO,România,I. C. Brătianu,NaN,Cluj-Napoca,Cluj,"[I. C. Brătianu (Universității), Cluj-Napoca]",Universității,NaN,4d3fc9d2c1d4721e2fa70ac7
3,Coffee You Go,Café,46.768914,23.590781,"[{'label': 'display', 'lat': 46.76891396084203...",81,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53106fce498e43148b52385b
4,Tucano Coffee,Coffee Shop,46.769859,23.593079,"[{'label': 'display', 'lat': 46.76985903899314...",244,RO,România,B-dul Eroilor nr. 19,400000,Cluj-Napoca,Cluj,"[B-dul Eroilor nr. 19, 400000 Cluj-Napoca]",NaN,Centru,585a3a790593e17ca0f5480a
5,Meron - Home of Coffee,Coffee Shop,46.768293,23.589267,"[{'label': 'display', 'lat': 46.76829295132707...",131,RO,România,Str. Napoca nr. 3,900329,Cluj-Napoca,Cluj,"[Str. Napoca nr. 3, 900329 Cluj-Napoca]",NaN,NaN,56641db4498e87cb76ed8837
6,Coffee Addicts,Coffee Shop,46.771816,23.589544,"[{'label': 'display', 'lat': 46.77181588247085...",273,RO,România,Ferdinand Street no 9,400000,Cluj-Napoca,Cluj,"[Ferdinand Street no 9, 400000 Cluj-Napoca]",NaN,NaN,56c4aaf7cd10703f0b5defed
7,Coffee 2 Go,Coffee Shop,46.769740,23.585265,"[{'label': 'display', 'lat': 46.7697396579221,...",359,RO,România,EMIL ISAC,NaN,Cluj-Napoca,Cluj,"[EMIL ISAC (Emil Isac), Cluj-Napoca]",Emil Isac,NaN,574ef6b3498e0e7c7626d67a
8,Coffee Addicts,Coffee Shop,46.771757,23.589818,"[{'label': 'display', 'lat': 46.771757, 'lng':...",264,RO,România,Regele Ferdinand 9,400110,Cluj-Napoca,Cluj,"[Regele Ferdinand 9, 400110 Cluj-Napoca]",NaN,NaN,59520841fd16bb5b31aa1053
9,Coffee To Go,Coffee Shop,46.767489,23.585886,"[{'label': 'display', 'lat': 46.76748861189874...",374,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5236d656498ebc888d2d3eea


In [18]:
dataframe_filtered_yellow = dataframe_filtered_yellow.drop(["labeledLatLngs", "cc", "country", "crossStreet", "postalCode", "state"], axis=1)
dataframe_filtered_yellow

,name,categories,lat,lng,distance,address,city,formattedAddress,neighborhood,id
0,Snack & Coffee,Café,46.771359,23.599048,727,NaN,NaN,NaN,NaN,51345ecbe4b041d1dfeded40
1,Let's Coffee,Coffee Shop,46.771115,23.585029,422,"Str. Constantin Daicoviciu, nr. 8",Cluj-Napoca,"[Str. Constantin Daicoviciu, nr. 8, 400020 Clu...",NaN,5603f924498ec2890ea4b4da
2,The Coffee House,Bar,46.769398,23.590663,54,I. C. Brătianu,Cluj-Napoca,"[I. C. Brătianu (Universității), Cluj-Napoca]",NaN,4d3fc9d2c1d4721e2fa70ac7
3,Coffee You Go,Café,46.768914,23.590781,81,NaN,NaN,NaN,NaN,53106fce498e43148b52385b
4,Tucano Coffee,Coffee Shop,46.769859,23.593079,244,B-dul Eroilor nr. 19,Cluj-Napoca,"[B-dul Eroilor nr. 19, 400000 Cluj-Napoca]",Centru,585a3a790593e17ca0f5480a
5,Meron - Home of Coffee,Coffee Shop,46.768293,23.589267,131,Str. Napoca nr. 3,Cluj-Napoca,"[Str. Napoca nr. 3, 900329 Cluj-Napoca]",NaN,56641db4498e87cb76ed8837
6,Coffee Addicts,Coffee Shop,46.771816,23.589544,273,Ferdinand Street no 9,Cluj-Napoca,"[Ferdinand Street no 9, 400000 Cluj-Napoca]",NaN,56c4aaf7cd10703f0b5defed
7,Coffee 2 Go,Coffee Shop,46.769740,23.585265,359,EMIL ISAC,Cluj-Napoca,"[EMIL ISAC (Emil Isac), Cluj-Napoca]",NaN,574ef6b3498e0e7c7626d67a
8,Coffee Addicts,Coffee Shop,46.771757,23.589818,264,Regele Ferdinand 9,Cluj-Napoca,"[Regele Ferdinand 9, 400110 Cluj-Napoca]",NaN,59520841fd16bb5b31aa1053
9,Coffee To Go,Coffee Shop,46.767489,23.585886,374,NaN,NaN,NaN,NaN,5236d656498ebc888d2d3eea


In [19]:
# create map of Cluj-Napoca - yellow zone using latitude and longitude values
cluj_yellow = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(dataframe_filtered_yellow['lat'], dataframe_filtered_yellow['lng'], dataframe_filtered_yellow['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.Circle([latitude, longitude],
                    radius=750).add_to(cluj_yellow)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cluj_yellow)  
    
cluj_yellow

### The search for green zone

In [20]:
search_query_green = 'Coffee' #searching Coffee places
radius = 1250 #1250m radius - our green zone
print(search_query_green + ' .... OK!')

Coffee .... OK!


In [21]:
url_green = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query_green, radius, LIMIT)
url_green

'https://api.foursquare.com/v2/venues/search?client_id=1YMM20J1EKW1H02USIKKCIKEGHOZRXP5COVZXO0RFAD2ITR2&client_secret=FZXBPCH0NTC232JEB5RCGACU053PCD2LQXRUF52SEUJOXFRB&ll=46.769379,23.5899542&oauth_token=XWJKBPN0TKL4LDEQQ5IXODS0I2SWULMCKBZKWRRKT1IOXWYM&v=20180604&query=Coffee&radius=1250&limit=100'

In [22]:
results_green = requests.get(url_green).json()
results_green

{'meta': {'code': 200, 'requestId': '60caec2a7348a6419a4da213'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '51345ecbe4b041d1dfeded40',
    'name': 'Snack & Coffee',
    'location': {'lat': 46.77135910503546,
     'lng': 23.599048118980512,
     'labeledLatLngs': [{'label': 'display',
       'lat': 46.77135910503546,
       'lng': 23.599048118980512}],
     'distance': 727,
     'cc': 'RO',
     'country': 'România'},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1623911466',
    'hasPerk': False},
   {'id': '54cb796a498eeb9f4132e7a6',
    'name': "Phil's Coffee Shop",
    'location': {'address': 'Str. Dorobanților nr. 21',
     'crossStreet': 'Str. Pitești',
     'lat': 46.77187563259055

In [23]:
# assign relevant part of JSON to venues
venues_green = results_green['response']['venues']

# tranform venues into a dataframe
dataframe_green = json_normalize(venues_green)
dataframe_green.head()

<ipython-input-23-ef720074eaea>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe_green = json_normalize(venues_green)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.address,location.crossStreet,location.postalCode,location.city,location.state,location.formattedAddress,location.neighborhood,venuePage.id
0,51345ecbe4b041d1dfeded40,Snack & Coffee,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1623911466,False,46.771359,23.599048,"[{'label': 'display', 'lat': 46.77135910503546...",727,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,54cb796a498eeb9f4132e7a6,Phil's Coffee Shop,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1623911466,False,46.771876,23.601388,"[{'label': 'display', 'lat': 46.77187563259055...",915,RO,România,Str. Dorobanților nr. 21,Str. Pitești,400118,Cluj-Napoca,Cluj,"[Str. Dorobanților nr. 21 (Str. Pitești), 4001...",NaN,NaN
2,5603f924498ec2890ea4b4da,Let's Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1623911466,False,46.771115,23.585029,"[{'label': 'display', 'lat': 46.77111478612739...",422,RO,România,"Str. Constantin Daicoviciu, nr. 8",NaN,400020,Cluj-Napoca,Cluj,"[Str. Constantin Daicoviciu, nr. 8, 400020 Clu...",NaN,NaN
3,4d3fc9d2c1d4721e2fa70ac7,The Coffee House,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1623911466,False,46.769398,23.590663,"[{'label': 'display', 'lat': 46.769398, 'lng':...",54,RO,România,I. C. Brătianu,Universității,NaN,Cluj-Napoca,Cluj,"[I. C. Brătianu (Universității), Cluj-Napoca]",NaN,NaN
4,53106fce498e43148b52385b,Coffee You Go,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1623911466,False,46.768914,23.590781,"[{'label': 'display', 'lat': 46.76891396084203...",81,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
dataframe_green.shape

(43, 19)

###### !we see that our green zone holds a total of 43 venues, but it includes the ones also included in the red  and yellow zone, we will need to sort that out later when we create the overall map for this project!

In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_green = ['name', 'categories'] + [col for col in dataframe_green.columns if col.startswith('location.')] + ['id']
dataframe_filtered_green = dataframe_green.loc[:, filtered_columns_green]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_green['categories'] = dataframe_filtered_green.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_green.columns = [column.split('.')[-1] for column in dataframe_filtered_green.columns]

dataframe_filtered_green

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,address,crossStreet,postalCode,city,state,formattedAddress,neighborhood,id
0,Snack & Coffee,Café,46.771359,23.599048,"[{'label': 'display', 'lat': 46.77135910503546...",727,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51345ecbe4b041d1dfeded40
1,Phil's Coffee Shop,Coffee Shop,46.771876,23.601388,"[{'label': 'display', 'lat': 46.77187563259055...",915,RO,România,Str. Dorobanților nr. 21,Str. Pitești,400118,Cluj-Napoca,Cluj,"[Str. Dorobanților nr. 21 (Str. Pitești), 4001...",NaN,54cb796a498eeb9f4132e7a6
2,Let's Coffee,Coffee Shop,46.771115,23.585029,"[{'label': 'display', 'lat': 46.77111478612739...",422,RO,România,"Str. Constantin Daicoviciu, nr. 8",NaN,400020,Cluj-Napoca,Cluj,"[Str. Constantin Daicoviciu, nr. 8, 400020 Clu...",NaN,5603f924498ec2890ea4b4da
3,The Coffee House,Bar,46.769398,23.590663,"[{'label': 'display', 'lat': 46.769398, 'lng':...",54,RO,România,I. C. Brătianu,Universității,NaN,Cluj-Napoca,Cluj,"[I. C. Brătianu (Universității), Cluj-Napoca]",NaN,4d3fc9d2c1d4721e2fa70ac7
4,Coffee You Go,Café,46.768914,23.590781,"[{'label': 'display', 'lat': 46.76891396084203...",81,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53106fce498e43148b52385b
5,Tucano Coffee,Coffee Shop,46.769859,23.593079,"[{'label': 'display', 'lat': 46.76985903899314...",244,RO,România,B-dul Eroilor nr. 19,NaN,400000,Cluj-Napoca,Cluj,"[B-dul Eroilor nr. 19, 400000 Cluj-Napoca]",Centru,585a3a790593e17ca0f5480a
6,Meron - Home of Coffee,Coffee Shop,46.768293,23.589267,"[{'label': 'display', 'lat': 46.76829295132707...",131,RO,România,Str. Napoca nr. 3,NaN,900329,Cluj-Napoca,Cluj,"[Str. Napoca nr. 3, 900329 Cluj-Napoca]",NaN,56641db4498e87cb76ed8837
7,Coffee Addicts,Coffee Shop,46.771816,23.589544,"[{'label': 'display', 'lat': 46.77181588247085...",273,RO,România,Ferdinand Street no 9,NaN,400000,Cluj-Napoca,Cluj,"[Ferdinand Street no 9, 400000 Cluj-Napoca]",NaN,56c4aaf7cd10703f0b5defed
8,Coffee 2 Go,Coffee Shop,46.769740,23.585265,"[{'label': 'display', 'lat': 46.7697396579221,...",359,RO,România,EMIL ISAC,Emil Isac,NaN,Cluj-Napoca,Cluj,"[EMIL ISAC (Emil Isac), Cluj-Napoca]",NaN,574ef6b3498e0e7c7626d67a
9,Coffee Addicts,Coffee Shop,46.771757,23.589818,"[{'label': 'display', 'lat': 46.771757, 'lng':...",264,RO,România,Regele Ferdinand 9,NaN,400110,Cluj-Napoca,Cluj,"[Regele Ferdinand 9, 400110 Cluj-Napoca]",NaN,59520841fd16bb5b31aa1053


In [26]:
# create map of Cluj-Napoca - green zone using latitude and longitude values
cluj_green = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(dataframe_filtered_green['lat'], dataframe_filtered_green['lng'], dataframe_filtered_green['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.Circle([latitude, longitude],
                    radius=1250
                   ).add_to(cluj_green)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cluj_green)  
    
cluj_green

## Test Search Query

In [27]:
search_query_test = 'Coffee' #searching Coffee places
radius = 5000 #5000m radius - our test zone
print(search_query_test + ' .... OK!')

Coffee .... OK!


In [28]:
url_test = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query_test, radius, LIMIT)
url_test

'https://api.foursquare.com/v2/venues/search?client_id=1YMM20J1EKW1H02USIKKCIKEGHOZRXP5COVZXO0RFAD2ITR2&client_secret=FZXBPCH0NTC232JEB5RCGACU053PCD2LQXRUF52SEUJOXFRB&ll=46.769379,23.5899542&oauth_token=XWJKBPN0TKL4LDEQQ5IXODS0I2SWULMCKBZKWRRKT1IOXWYM&v=20180604&query=Coffee&radius=5000&limit=100'

In [29]:
results_test = requests.get(url_test).json()
results_test

{'meta': {'code': 200, 'requestId': '60caec7e5566da5ba0d5d866'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '51345ecbe4b041d1dfeded40',
    'name': 'Snack & Coffee',
    'location': {'lat': 46.77135910503546,
     'lng': 23.599048118980512,
     'labeledLatLngs': [{'label': 'display',
       'lat': 46.77135910503546,
       'lng': 23.599048118980512}],
     'distance': 727,
     'cc': 'RO',
     'country': 'România'},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1623911550',
    'hasPerk': False},
   {'id': '54cb796a498eeb9f4132e7a6',
    'name': "Phil's Coffee Shop",
    'location': {'address': 'Str. Dorobanților nr. 21',
     'crossStreet': 'Str. Pitești',
     'lat': 46.77187563259055

In [30]:
# assign relevant part of JSON to venues
venues_test = results_test['response']['venues']

# tranform venues into a dataframe
dataframe_test = json_normalize(venues_test)
dataframe_test.head()

<ipython-input-30-51e8e0b0b0d7>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe_test = json_normalize(venues_test)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.address,location.crossStreet,location.postalCode,location.city,location.state,location.formattedAddress,location.neighborhood,venuePage.id
0,51345ecbe4b041d1dfeded40,Snack & Coffee,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1623911550,False,46.771359,23.599048,"[{'label': 'display', 'lat': 46.77135910503546...",727,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,54cb796a498eeb9f4132e7a6,Phil's Coffee Shop,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1623911550,False,46.771876,23.601388,"[{'label': 'display', 'lat': 46.77187563259055...",915,RO,România,Str. Dorobanților nr. 21,Str. Pitești,400118,Cluj-Napoca,Cluj,"[Str. Dorobanților nr. 21 (Str. Pitești), 4001...",NaN,NaN
2,5603f924498ec2890ea4b4da,Let's Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1623911550,False,46.771115,23.585029,"[{'label': 'display', 'lat': 46.77111478612739...",422,RO,România,"Str. Constantin Daicoviciu, nr. 8",NaN,400020,Cluj-Napoca,Cluj,"[Str. Constantin Daicoviciu, nr. 8, 400020 Clu...",NaN,NaN
3,4d3fc9d2c1d4721e2fa70ac7,The Coffee House,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1623911550,False,46.769398,23.590663,"[{'label': 'display', 'lat': 46.769398, 'lng':...",54,RO,România,I. C. Brătianu,Universității,NaN,Cluj-Napoca,Cluj,"[I. C. Brătianu (Universității), Cluj-Napoca]",NaN,NaN
4,53106fce498e43148b52385b,Coffee You Go,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1623911550,False,46.768914,23.590781,"[{'label': 'display', 'lat': 46.76891396084203...",81,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
dataframe_test.shape

(50, 19)

###### !we see that our test zone holds a total of 50 venues, but it includes the ones also included in the red, yellow and green zones, we will need to sort that out later when we create the overall map for this project!

In [32]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_test = ['name', 'categories'] + [col for col in dataframe_test.columns if col.startswith('location.')] + ['id']
dataframe_filtered_test = dataframe_test.loc[:, filtered_columns_test]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_test['categories'] = dataframe_filtered_test.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_test.columns = [column.split('.')[-1] for column in dataframe_filtered_test.columns]

dataframe_filtered_test

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,address,crossStreet,postalCode,city,state,formattedAddress,neighborhood,id
0,Snack & Coffee,Café,46.771359,23.599048,"[{'label': 'display', 'lat': 46.77135910503546...",727,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51345ecbe4b041d1dfeded40
1,Phil's Coffee Shop,Coffee Shop,46.771876,23.601388,"[{'label': 'display', 'lat': 46.77187563259055...",915,RO,România,Str. Dorobanților nr. 21,Str. Pitești,400118,Cluj-Napoca,Cluj,"[Str. Dorobanților nr. 21 (Str. Pitești), 4001...",NaN,54cb796a498eeb9f4132e7a6
2,Let's Coffee,Coffee Shop,46.771115,23.585029,"[{'label': 'display', 'lat': 46.77111478612739...",422,RO,România,"Str. Constantin Daicoviciu, nr. 8",NaN,400020,Cluj-Napoca,Cluj,"[Str. Constantin Daicoviciu, nr. 8, 400020 Clu...",NaN,5603f924498ec2890ea4b4da
3,The Coffee House,Bar,46.769398,23.590663,"[{'label': 'display', 'lat': 46.769398, 'lng':...",54,RO,România,I. C. Brătianu,Universității,NaN,Cluj-Napoca,Cluj,"[I. C. Brătianu (Universității), Cluj-Napoca]",NaN,4d3fc9d2c1d4721e2fa70ac7
4,Coffee You Go,Café,46.768914,23.590781,"[{'label': 'display', 'lat': 46.76891396084203...",81,RO,România,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53106fce498e43148b52385b
5,Tucano Coffee,Coffee Shop,46.769859,23.593079,"[{'label': 'display', 'lat': 46.76985903899314...",244,RO,România,B-dul Eroilor nr. 19,NaN,400000,Cluj-Napoca,Cluj,"[B-dul Eroilor nr. 19, 400000 Cluj-Napoca]",Centru,585a3a790593e17ca0f5480a
6,Meron - Home of Coffee,Coffee Shop,46.768293,23.589267,"[{'label': 'display', 'lat': 46.76829295132707...",131,RO,România,Str. Napoca nr. 3,NaN,900329,Cluj-Napoca,Cluj,"[Str. Napoca nr. 3, 900329 Cluj-Napoca]",NaN,56641db4498e87cb76ed8837
7,Coffee Addicts,Coffee Shop,46.771816,23.589544,"[{'label': 'display', 'lat': 46.77181588247085...",273,RO,România,Ferdinand Street no 9,NaN,400000,Cluj-Napoca,Cluj,"[Ferdinand Street no 9, 400000 Cluj-Napoca]",NaN,56c4aaf7cd10703f0b5defed
8,Coffee 2 Go,Coffee Shop,46.769740,23.585265,"[{'label': 'display', 'lat': 46.7697396579221,...",359,RO,România,EMIL ISAC,Emil Isac,NaN,Cluj-Napoca,Cluj,"[EMIL ISAC (Emil Isac), Cluj-Napoca]",NaN,574ef6b3498e0e7c7626d67a
9,Coffee Addicts,Coffee Shop,46.771757,23.589818,"[{'label': 'display', 'lat': 46.771757, 'lng':...",264,RO,România,Regele Ferdinand 9,NaN,400110,Cluj-Napoca,Cluj,"[Regele Ferdinand 9, 400110 Cluj-Napoca]",NaN,59520841fd16bb5b31aa1053


In [33]:
# create map of Cluj-Napoca - test zone using latitude and longitude values
cluj_test = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(dataframe_filtered_test['lat'], dataframe_filtered_test['lng'], dataframe_filtered_test['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.Circle([latitude, longitude],
                    radius=5000
                   ).add_to(cluj_test)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cluj_test)  
    
cluj_test

## Counting the unique values in our data frames on the column Category 

In [34]:
print(dataframe_filtered_test['categories'].value_counts())

Coffee Shop           32
Café                   9
Bar                    3
Chocolate Shop         1
Casino                 1
College Rec Center     1
Beach Bar              1
Pub                    1
Cocktail Bar           1
Name: categories, dtype: int64


In [35]:
print(dataframe_filtered_green['categories'].value_counts())

Coffee Shop           29
Café                   7
Bar                    2
Chocolate Shop         1
Casino                 1
College Rec Center     1
Beach Bar              1
Cocktail Bar           1
Name: categories, dtype: int64


In [36]:
print(dataframe_filtered_yellow['categories'].value_counts())

Coffee Shop           25
Café                   7
Bar                    1
College Rec Center     1
Cocktail Bar           1
Name: categories, dtype: int64


In [37]:
print(dataframe_filtered_red['categories'].value_counts())

Coffee Shop     17
Café             7
Bar              1
Cocktail Bar     1
Name: categories, dtype: int64


###### The value_counts method give us an interesting insight: turns out not all Caffee Houses are just about Coffee! It is interesting to see that some Bars and even a Casino choose to be tagged as Coffee Place. This enforces the ideeathat there mai be room for new venues!

###### For better visualisation we normalised the data:

In [38]:
dataframe_filtered_test['categories'].value_counts(normalize=True)

Coffee Shop           0.64
Café                  0.18
Bar                   0.06
Chocolate Shop        0.02
Casino                0.02
College Rec Center    0.02
Beach Bar             0.02
Pub                   0.02
Cocktail Bar          0.02
Name: categories, dtype: float64

In [39]:
dataframe_filtered_green['categories'].value_counts(normalize=True)

Coffee Shop           0.674419
Café                  0.162791
Bar                   0.046512
Chocolate Shop        0.023256
Casino                0.023256
College Rec Center    0.023256
Beach Bar             0.023256
Cocktail Bar          0.023256
Name: categories, dtype: float64

In [40]:
dataframe_filtered_yellow['categories'].value_counts(normalize=True)

Coffee Shop           0.714286
Café                  0.200000
Bar                   0.028571
College Rec Center    0.028571
Cocktail Bar          0.028571
Name: categories, dtype: float64

In [41]:
dataframe_filtered_red['categories'].value_counts(normalize=True)

Coffee Shop     0.653846
Café            0.269231
Bar             0.038462
Cocktail Bar    0.038462
Name: categories, dtype: float64

###### Calculating density of Coffee Places in each of the data frames

In [68]:
dens_test = dataframe_filtered_test["name"].count()/78.5 #78.5 is area of circle with radius 5000 m
print("Density of Coffe Houses in Cluj-Napoca is: ", dens_test, " venues/square km")

Density of Coffe Houses in Cluj-Napoca is:  0.6369426751592356  venues/square km


In [69]:
dens_green = dataframe_filtered_green["name"].count()/4.2 # 4.2 is area of circle with radius 1250 m
print("Density of Coffe Houses in green_zone in Cluj-Napoca is: ", dens_green, " venues/square km")

Density of Coffe Houses in green_zone in Cluj-Napoca is:  10.238095238095237  venues/square km


In [70]:
dens_yellow = dataframe_filtered_yellow["name"].count()/1.75 # is area of circle with radius 750 m
print("Density of Coffe Houses in yellow_zone in Cluj-Napoca is: ", dens_yellow, " venues/square km")

Density of Coffe Houses in yellow_zone in Cluj-Napoca is:  20.0  venues/square km


In [71]:
dens_red = dataframe_filtered_red["name"].count()/78.5 # is area of circle with radius 500 m
print("Density of Coffe Houses in red_zone in Cluj-Napoca is: ", dens_red, " venues/square km")

Density of Coffe Houses in red_zone in Cluj-Napoca is:  0.33121019108280253  venues/square km
